## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = ''

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
!pip install -r path/requirements.txt

#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json

# 더 필요한 라이브러리 추가 -------------

### (3) OpenAI API Key 환경 변수 설정

* 제공받은 open ai api key를 **api_key.txt** 파일에 저장

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 결과는 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2. 미션1 : STT**

### (1) 제공된 데이터 변환
* 세부사항
    * 사용 모델 : whisper-1
    * 제공 받은 오디오 파일을 읽어서 텍스트로 변환시켜 봅시다.
        * 반복문을 통해 파일 하나씩 읽어서 텍스트 변환
        * 변환된 텍스트를 데이터 프레임에 추가

|filename|text|
|----|----|
|audio3.mp3|어쩌구 저쩌구...급해요.|

* 음성파일 변환

In [ ]:
# 음성파일 경로 지정
audio_path = ''

In [ ]:
# OpenAI 클라이언트 생성
client = OpenAI()

In [ ]:
# 위스퍼 모델 사용 : 제공된 음성파일 중 1개를 텍스트로 변환해보기
filename = 'audio2.mp3'
audio_file = open(audio_path + filename, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
)

print(transcript, type(transcript))

119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>


* 음성파일 변환 함수 생성

In [ ]:
def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",)

    # 결과 반환
    print(transcript, type(transcript))
    return transcript

In [ ]:
# 음성파일 이름을 리스트에 담기
file_names = [f for f in os.listdir(audio_path) if os.path.isfile(os.path.join(audio_path, f))]
print(file_names)

['audio1.mp3', 'audio3.mp3', 'audio2.mp3', 'audio4.mp3', 'audio5.mp3', '651e464d69a4f266f0626a0a_20220101.wav', '651e464d69a4f266f0626a00_20220101.wav', '651e464d69a4f266f0626a4f_20220101.wav', '651e464d69a4f266f0626a4e_20220101.wav', '651e464d69a4f266f0626a4c_20220101.wav', '651e464d69a4f266f0626a2c_20220101.wav', '651e464d69a4f266f0626a1f_20220101.wav', '651e464d69a4f266f0626a2a_20220101.wav', '651e464d69a4f266f0626a3e_20220101.wav', '651e464d69a4f266f0626a01_20220101.wav', '651e464d69a4f266f0626a3d_20220101.wav', '651e464d69a4f266f0626a0d_20220101.wav', '651e464d69a4f266f0626a3b_20220101.wav', '651e464d69a4f266f0626a0b_20220101.wav', '651e464d69a4f266f0626a5e_20220101.wav', '651e464d69a4f266f0626a5c_20220101.wav', '651e464d69a4f266f0626a4d_20220101.wav', '651e464d69a4f266f0626a5a_20220101.wav', '651e464d69a4f266f0626a4b_20220101.wav', '651e464d69a4f266f0626a3f_20220101.wav', '651e464d69a4f266f0626a04_20220101.wav', '651e464d69a4f266f0626a2d_20220101.wav', '651e464d69a4f266f0626a2f_2

In [ ]:
# 반복문을 통해, 파일 하나씩 읽어서 텍스트 변환, 변환된 텍스트를 데이터 프레임에 추가

# 빈 데이터프레임 선언
df = pd.DataFrame(columns=['file_name','transcription'])

# 반복문 수행하면서 오디오 변환
for i, file_name in enumerate(file_names):
    transcript = audio_to_text(audio_path, file_name)
    df.loc[len(df)] = {'file_name': file_name, 'transcription': transcript}

# 데이터프레임 결과 조회
print(df)

지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.
 <class 'str'>
동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안 빠져요. 어떻게 해야 할까요? 동생이 너무 힘들어 하네요.
 <class 'str'>
119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 아프고 좀 띵한 것 같아요. 우한이 좀 들어요. 어떻게 해야 할까요?
 <class 'str'>
아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리가 어지럽고 속이 매스꺼워요. 어떻게 해야 할까요?
 <class 'str'>
화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈 것 같은데 응급실을 가야 할까요?
 <class 'str'>
여보세요 119에요 여보세요 예 119에요 예 119라고요 119 예 어디 아프세요? 아 다 아파요 예? 다 아프다고요 주소가 어떻게 돼요? 모르세요 이 얘기는 예? 모르겠다고요 주소를 알아야 가죠 아 저도 잘 모르지 지금 여기가 집 아니에요? 예 밖에에요? 안에 안에 그러니까요 지금 집 어디 뭐 나와서 혹시 말 못해요? 예 위치를 알아야 어디라도 가죠 아 모르겠어요 주소 좀 생각해보세요 주소 찾아서 빨리 가게 안그러면 빨리 못간다니까요 아 모르겠어요 주소를 알려주셔야 빨리 가죠 주소가 어떻게 돼요? 잘 모르겠어요 주소를 모른다고 하면 안된다니까요
 <class 'str'>
여보세요. 엘리구입니다. 친구가 지금 공황장애가 심하게 와서 숨이 안 쉬어진다고 지금 약 먹었는데. 네, 주소 알려주세요. 의료 상담이 필요하세요? 구급차 필요하세요? 주소, 주소. 여기 그 목동사거리 옛날에 동방줄 서 있다가. 건물 밖으로 

In [ ]:
df

,file_name,transcription
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
1,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
2,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...
...,...,...
57,651e464d69a4f266f0626a7b_20220101.wav,네 뭐 좀 여쭤보려고 그러는데요. 산모인데요. 아직 임신 초기고요. 의료 상담이시면...
58,651e464d69a4f266f0626a07_20220101.wav,"무슨 일이세요? 네, 저기 그... 집에 어머님이 계시는데요. 네네. 오늘 지금 열..."
59,651e464d69a4f266f0626a6f_20220101.wav,"무슨 일이에요? 여보세요? 119에요. 네, 선생님. 안녕하세요. 저기 열이 심한 ..."
60,651e464d69a4f266f0626a6d_20220101.wav,119입니다. 119입니다. 말씀하세요. 예. 다름이 아니고 저희 어르신이 고열나면...


## **3. 미션2 : Summary**

* 세부사항
    * 문서요약 예제 파일을 참조하여 테스트 해 봅니다.
    * 코드를 참조하여, 원하는 형식에 맞게 요약이 되도록 프롬프트를 구성합니다.
        * 요약 시 중요 키워드들이 함께 도출되도록 합니다.
        * 가능하다면, 요약 문장 길이에 제한을 둡시다.
    * 반복문을 통해 요약하고, 결과를 데이터프레임에 추가합니다.
        * summary 열을 추가하고, 요약 결과를 입력
            * 요약결과와 키워드는 하나의 문자열로 붙여서 summary열에 추가

### (1) 문서 요약

* 문서 요약 함수로 생성

In [ ]:
def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정

    system_role = "당신은 응급상황에서 어떠한 상황인지 파악하고, 증상을 간단한 키워드로 요약하는 어시스턴트입니다. 응답은 다음의 형식을 지켜주세요 {\"summary\": \"텍스트 요약\", \"부위\": \"다친 부위\", \"증상\": \"관련 증상\"}"

    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답 받기
    print(response)

    # 응답형식을 정리하고 return
    return response.choices[0].message.content

* 저장된 text를 하나씩 불러와서 요약하고 다시 저장하기

In [ ]:
response = []
for i in range(len(df)):
    input_text = df['transcription'][i]
    response.append(text_summary(input_text))

ChatCompletion(id='chatcmpl-AUnqkvhsBaJMExt3OwHDOhF4r3tQT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"summary": "머리에서 출혈이 있는 넘어짐으로 인한 응급상황", "부위": "머리", "증상": "머리에서 피가 나고 어지러움"}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731904826, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=296, total_tokens=361, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletion(id='chatcmpl-AUnqloxEodTc9xFvmUPrGegUd1WnM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"summary": "콩이 코에 들어간 상황", "부위": "코", "증상": "한쪽 코가 막힘"}', refusal=None, role='assistant', 

In [ ]:
response

['{"summary": "머리에서 출혈이 있는 넘어짐으로 인한 응급상황", "부위": "머리", "증상": "머리에서 피가 나고 어지러움"}',
 '{"summary": "콩이 코에 들어간 상황", "부위": "코", "증상": "한쪽 코가 막힘"}',
 '{"summary": "열이 올라 38도, 머리 통증, 띵함, 우한 느낌", "부위": "머리", "증상": "열이 올라 머리 통증, 띵함, 우한 느낌"}',
 '{"summary": "머리에 충격을 받은 후 어지러움과 메스꺼움 증세 보임", "부위": "머리", "증상": "어지러움, 메스꺼움"}',
 '{"summary": "미끄러져 엉덩방아를 찍고 꼬리뼈가 계속 아프다.", "부위": "엉덩이, 꼬리뼈", "증상": "엉덩이 통증, 꼬리뼈 통증"}',
 '{"summary": "환자가 현장 주소를 알려주지 못하고 혼란스러워하는 상황", "부위": "정확한 부위 미상", "증상": "의식 불명 또는 혼란"}  ',
 '{"summary": "친구가 공황장애로 숨쉬기 어려워하고 있으며 약을 복용함", "부위": "호흡기", "증상": "숨쉬기 어려움, 공황장애"}',
 '{"summary": "어머니가 몸살 비슷한 증상으로 힘들어하고 물을 잘 못 마심", "부위": "전신", "증상": "몸살, 수분 섭취 어려움"}',
 '{"summary": "술을 과다섭취하고 의식을 잃은 환자가 쓰러져 있으며 정신이 없어하고 토를 하고 있습니다. 남자분", "부위": "머리", "증상": "의식 잃음, 토 오름, 숨쉬기 어려움"}',
 '{"summary": "소변이 잘 나오지 않는 증상과 감기 증상이 있는 환자", "부위": "배 및 호흡기", "증상": "소변이 잘 나오지 않음, 감기 증상"}',
 '{"summary": "어머니가 오른쪽 팔을 못 움직이고 있는 상황", "부위": "팔", "증상": "운동이 제한되고 힘을 쓰기 어려움"}',
 '{"summary": "화이자 백신 접종 후 식은땀과 기운이 없음"

### (2) 전국 병원 응급실 정보 수집



#### 1) 인증키 발급

* 인증키 발급 절차
    * 1) data.go.kr 회원가입
    * 2) 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
https://www.data.go.kr/data/15000563/openapi.do 로 이동
    * 3) 활용신청
        * 활용목적 : 기타(개인 학습 용도)
        * 상세 기능선택
            * 응급의료기관 목록정보 조회
            * 응급의료기관 위치정보 조회
            * 응급의료기관 기본정보 조회
    * 4) 인증키 확인
        * 마이페이지 > Open API > 활용신청현황
        * [승인] 국립중앙의료원_전국 응급의료기관 정보 조회 서비스
        * 일반 인증키(Decoding) 이용

#### 2) 데이터 수집

In [ ]:
# path 확인
path

'/content/drive/MyDrive/KT_aivle/미니프로젝트/2024.11.18_미니프로젝트_6차_4일차_실습자료/'

In [ ]:
# 응급실 데이터 수집하기
# 응급실 목록정보 조회

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire'
serviceKey = ''    # 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName') #기관명
    duty_addr = item.findtext('dutyAddr') #주소
    # 필요한 정보 추가
    latitude = item.findtext('wgs84Lat')
    longitude = item.findtext('wgs84Lon')

    # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)
    data.append({'기관명' : duty_name, '기관주소': duty_addr, '위도' : latitude, '경도': longitude})

# 데이터프레임으로 변환
df_basic =  pd.DataFrame(data)

In [ ]:
df_basic

,기관명,기관주소,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),35.54823820112527,129.30701143429678
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,35.23602946449906,129.21649161387128
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",37.5089936801167,126.669478649026
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",37.234641294534285,127.21049868474802
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),37.240316373,127.2144914405
...,...,...,...,...
521,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",37.39340413136221,126.92447734066778
522,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",37.35864464913468,126.94735972779895
523,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),35.18541271514478,129.12145899191526
524,흑룡의원,인천광역시 옹진군 백령면 백령로 831,37.959524356,124.6654985764


In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
import time

In [ ]:
geolocator = Nominatim(user_agent='chiricuto')

# 주소에서 위도와 경도를 찾는 함수
def get_coordinates(address):
    try:
        simple_address = address.split(",")[0].split("(")[0] # 주소에서 상세한 부분을 제거하여 더 단순화
        location = geolocator.geocode(simple_address)
        if location is not None:
            return location.latitude, location.longitude
        else:
            print(f"주소를 찾을 수 없습니다: {address}")
            return None, None
    except Exception as e:
        print(f"오류 발생: {e}, 주소: {address}")
        return None, None

# 데이터프레임에 위도와 경도 추가
for i, row in df_basic.iterrows():
    latitude, longitude = get_coordinates(row['기관주소'])
    df_basic.loc[i, '위도'] = latitude
    df_basic.loc[i, '경도'] = longitude
    time.sleep(1)

# 결과 출력
print(df_basic)

주소를 찾을 수 없습니다: 경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)


주소를 찾을 수 없습니다: 경상북도 김천시 모암길 24 (모암동)


주소를 찾을 수 없습니다: 전라남도 구례군 구례읍 동편제길 4
주소를 찾을 수 없습니다: 전라남도 순천시 조례1길 24 (조례동)
주소를 찾을 수 없습니다: 전북특별자치도 김제시 서암4길 45 (서암동)
주소를 찾을 수 없습니다: 전라남도 보성군 미력면 가평길 36-17


주소를 찾을 수 없습니다: 경기도 부천시 오정구 소사로 726, 지하4층~지상5층 (원종동)
주소를 찾을 수 없습니다: 경상북도 상주시 상서문로 53 (남성동)
주소를 찾을 수 없습니다: 경상남도 함안군 칠원읍 용산2길 45-13


주소를 찾을 수 없습니다: 경기도 가평군 설악면 미사리로 267-177


주소를 찾을 수 없습니다: 전라남도 영암군 영암읍 오리정길 8
주소를 찾을 수 없습니다: 충청남도 예산군 예산읍 신례원로 26
주소를 찾을 수 없습니다: 경상북도 김천시 신음1길 12 (신음동)
주소를 찾을 수 없습니다: 전라남도 보성군 벌교읍 남하로 12
주소를 찾을 수 없습니다: 전라남도 신안군 비금면 송치길 155-11
주소를 찾을 수 없습니다: 충청북도 음성군 금왕읍 음성로1230번길 10
주소를 찾을 수 없습니다: 전라남도 순천시 우명길 42 (연향동)


주소를 찾을 수 없습니다: 전라남도 장성군 장성읍 역전로 171
주소를 찾을 수 없습니다: 경상남도 창원시 의창구 용동로57번길 8 (사림동)
주소를 찾을 수 없습니다: 강원특별자치도 철원군 갈말읍 명성로 208-0
주소를 찾을 수 없습니다: 충청북도 청주시 서원구 1순환로 776-0 (개신동,충북대학교병원)


주소를 찾을 수 없습니다: 경상남도 창녕군 창녕읍 교리1길 2
주소를 찾을 수 없습니다: 경기도 파주시 탄현면 평화로574번길 17, (의)청학의료재단 파주성모병원.요양병원
주소를 찾을 수 없습니다: 광주광역시 광산구 용아로 259 하남성심병원 (산정동)
주소를 찾을 수 없습니다: 울산광역시 동구 대학병원로 25, 울산대학교병원 (전하동)
주소를 찾을 수 없습니다: 강원특별자치도 홍천군 홍천읍 산림공원1길 17
                 기관명                                            기관주소  \
0    (의)내경의료재단울산제일병원                      울산광역시 남구 남산로354번길 26 (신정동)   
1      (의)서일의료재단기장병원                         부산광역시 기장군 기장읍 대청로72번길 6   
2    (의)성세의료재단 뉴성민병원  인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))   
3     (의)영문의료재단다보스병원       경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)   
4    (의)효심의료재단용인서울병원                        경기도 용인시 처인구 고림로 81 (고림동)   
..               ...                                             ...   
521      효산의료재단안양샘병원                  경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)   
522       효산의료재단지샘병원               경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)   
523           효성시티병원                       부산광역시 해운대구 해운대로 135 (재송동)   
524             흑룡의원                           인

In [ ]:
df_basic

,기관명,기관주소,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),35.548538,129.305706
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,35.234664,129.215478
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",37.509355,126.671129
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",NaN,NaN
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),37.245068,127.218638
...,...,...,...,...
521,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",37.397804,126.915755
522,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",37.372789,126.942672
523,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),35.177160,129.186879
524,흑룡의원,인천광역시 옹진군 백령면 백령로 831,37.974239,124.694594


In [ ]:
# 응급실 데이터 수집하기
# 응급실 기본정보 조회

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire'
serviceKey = ''

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName') #기관명
    duty_addr = item.findtext('dutyAddr') #주소
    # 필요한 정보 추가
    latitude = item.findtext('wgs84Lat')
    longitude = item.findtext('wgs84Lon')
    hvec = item.findtext('hvec')

    # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)
    data.append({'기관명' : duty_name, '기관주소': duty_addr, '위도' : latitude, '경도': longitude, '응급실':hvec})

# 데이터프레임으로 변환
df_em =  pd.DataFrame(data)

In [ ]:
df_em

,기관명,기관주소,위도,경도,응급실
0,경희대학교병원,서울특별시 동대문구 경희대로 23 (회기동),37.5938765502235,127.05183223390303,16
1,건국대학교병원,서울특별시 광진구 능동로 120-1 (화양동),37.54084479467721,127.0721229093036,6
2,중앙대학교병원,서울특별시 동작구 흑석로 102 (흑석동),37.50707428493414,126.96079378447554,15
3,순천향대학교 부속 서울병원,서울특별시 용산구 대사관로 59 (한남동),37.53384172231443,127.00441798640304,20
4,이화여자대학교의과대학부속목동병원,서울특별시 양천구 안양천로 1071 (목동),37.53654282637804,126.8862159683056,30
...,...,...,...,...,...
995,수연세안과의원,"서울특별시 서초구 서초대로77길 54, 서초더블유타워 5, 6층 (서초동)",37.502506415764955,127.02484127797528,None
996,서초좋은의원,"서울특별시 서초구 서초중앙로 238, 306호 (반포동, 삼호가든상가)",37.5029619493601,127.012070984447,None
997,베스탑비뇨기과의원,"서울특별시 송파구 올림픽로 269, 2층 221호 (신천동, 롯데캐슬골드)",37.5144273491608,127.100611434725,None
998,연세이김마취통증의학과의원,서울특별시 송파구 송파대로30길 5 (가락동),37.4944808900047,127.118048260052,None


In [ ]:
df_merged = pd.merge(df_basic, df_em, on=['기관명', '기관주소'], how='inner')
df_merged

,기관명,기관주소,위도,경도,응급실
0,가톨릭대학교여의도성모병원,"서울특별시 영등포구 63로 10, 여의도성모병원 (여의도동)",37.51827233800711,126.93673129599131,17
1,강남고려병원,서울특별시 관악구 관악로 242 (봉천동),37.4856188363947,126.956781703053,10
2,강남베드로병원,"서울특별시 강남구 남부순환로 2649, 베드로병원 (도곡동)",37.485612179925724,127.0395873429168,None
3,강남힐병원,"서울특별시 관악구 남부순환로 1449, 강남힐병원 (신림동)",37.4816513439454,126.911648078471,None
4,강동경희대학교의대병원,서울특별시 강동구 동남로 892 (상일동),37.5520459324005,127.157084787845,12
...,...,...,...,...,...
57,한림대학교한강성심병원,"서울특별시 영등포구 버드나루로7길 12 (영등포동7가, 한강성심병원)",37.52346674579277,126.91033000120589,30
58,한양대학교병원,서울특별시 성동구 왕십리로 222-1 (사근동),37.559944533564746,127.04488284061982,10
59,혜민병원,서울특별시 광진구 자양로 85 (자양동),37.535315660180416,127.08360130258502,10
60,홍익병원,"서울특별시 양천구 목동로 225, 홍익병원본관 (신정동)",37.52844147447355,126.8636640030062,8


In [ ]:
# csv 파일로 저장(인덱스 제외)
df_basic.to_csv('df_basic_true.csv', index=False )

In [ ]:
# 응급실 데이터 수집하기
# 응급실 기본정보 조회

url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'
serviceKey = ''    # 여러분의 일반 인증키(Decoding)

params = {
    'serviceKey': serviceKey,
    'pageNo': '1', 'numOfRows': '1000',  # 전체 응급실 수가 500여개 됨. 1000개면 충분
    'STAGE1' : '서울특별시', 'STAGE2':'강남구',
    'format': 'xml'
}

response = requests.get(url, params = params)

# 정상 수행 되었다면 200
print(response)

<Response [200]>


In [ ]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dutyName>연세대학교의과대학 강남세브란스병원</dutyName><dutyTel3>02-2019-3333</dutyTel3><hpid>A1100015</hpid><hv10>Y</hv10><hv11>Y</hv11><hv28>2</hv28><hv29>1</hv29><hv30>1</hv30><hv34>3</hv34><hv41>3</hv41><hv42>Y</hv42><hv5>Y</hv5><hv6>7</hv6><hv7>Y</hv7><hvamyn>Y</hvamyn><hvangioayn>Y</hvangioayn><hvcrrtayn>Y</hvcrrtayn><hvctayn>Y</hvctayn><hvec>-6</hvec><hvecmoayn>Y</hvecmoayn><hvgc>127</hvgc><hvhypoayn>Y</hvhypoayn><hvicc>11</hvicc><hvidate>20241118160826</hvidate><hvincuayn>Y</hvincuayn><hvmriayn>Y</hvmriayn><hvncc>5</hvncc><hvoc>0</hvoc><hvoxyayn>N1</hvoxyayn><hvs01>15</hvs01><hvs02>2</hvs02><hvs03>1</hvs03><hvs04>2</hvs04><hvs08>28</hvs08><hvs12>12</hvs12><hvs15>8</hvs15><hvs17>29</hvs17><hvs22>23</hvs22><hvs25>5</hvs25><hvs26>1</hvs26><hvs27>4</hvs27><hvs28>3</hvs28><hvs29>6</hvs29><hvs30>47</hvs30><hvs31>10</hvs31><hvs32>23</hv

In [ ]:
# response xml에서 주요 정보 찾기
root = ET.fromstring(response.text)

data = []

for item in root. findall('.//item'):
    duty_name = item.findtext('dutyName') #기관명
    # 필요한 정보 추가
    hvgc = item.findtext('hvgc') #입원실
    hvec = item.findtext('hvec') #응급실
    hvidate = item.findtext('hvidate') #입력일시
    # 빈 리스트 data에 딕시너리 형태({'칼럼이름':값, ...})로 저장(추가)
    data.append({'기관명' : duty_name, '응급실': hvec, '입원실': hvgc, '갱신일자': hvidate})

# 데이터프레임으로 변환
df_em =  pd.DataFrame(data)

In [ ]:
df_em

,기관명,응급실,입원실,갱신일자
0,연세대학교의과대학 강남세브란스병원,-6,127,20241118160826
1,삼성서울병원,-8,482,20241118161110
